In [ ]:
import loompy
import os
import glob

In [ ]:
# use loompy.combine(files, key="Accession") to combine all loom files in data/103.self_workflow/sample/velocyto/*.loom
# sample is N1, N2, N4, T1, T2, T4
# output is data/103.self_workflow/velocyto_combined.loom
# first get a list of loom file paths for all samples
samples = ["N1", "N2", "N4", "N7", "T1", "T2", "T4", "T7"]
files = []
for sample in samples:
    files.extend(glob.glob(f"data/103.self_workflow/{sample}/velocyto/*.loom"))

for f in files:
    sample = os.path.basename(f).split(".")[0]
    if sample != "N1":
        # extract the sample name from the file path, e.g. N1 from data/103.self_workflow/N1/velocyto/N1.loom
        with loompy.connect(f) as ds:
            new_cell_ids = []
            for cell_id in ds.ca.CellID:
                parts = cell_id.split(":")
                print(f"Processing sample: {sample}")
                print(f"Parts: {parts}")
                new_cell_id = f"{sample}_{parts[1]}"
                print(f"New cell_id: {new_cell_id}")
                # remove the last x in the cell id
                new_cell_id = new_cell_id[:-1]
                new_cell_ids.append(new_cell_id)

            ds.ca.CellID = new_cell_ids
            print(ds.ca.CellID)

loompy.combine(
    files, key="Accession", output_file=f"data/103.self_workflow/velocyto_combined.loom"
)

In [ ]:
files